# Giriş İşlemleri

In [ ]:
%%capture
!pip install kagglehub pandas datasets scikit-learn torch transformers[torch] matplotlib seaborn

In [ ]:
import kagglehub
import pandas as pd
import os
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np
import torch
import json

In [ ]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/spam_message_classifier_tr"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
model_save_path = os.path.join(kok_dizin, "egitilen_model")
model_cikti_dizini = os.path.join(kok_dizin, "kilometre_tasi")
model_log_dizini = os.path.join(kok_dizin, "log")
eval_sonuc_json_path = os.path.join(model_log_dizini, "eval_sonuc.json")
train_sonuc_json_path = os.path.join(model_log_dizini, "train_sonuc.json")
if not os.path.exists(model_cikti_dizini):
    os.makedirs(model_cikti_dizini)
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

In [ ]:
cihaz = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cihaz: {cihaz}")

# Veri Kümesi Okuma

## Veri kümesi okuma fonksiyonu

In [ ]:
def veri_kumesi_oku(veri_kumesi_adi: str) -> pd.DataFrame:
    """
    Kaggle veri kümesini indirir ve döndürür.

    Args:
        veri_kumesi_adi (str): Kaggle veri kümesinin yolu.
        gecici_yol (str): Geçici dosya yolu.
    Returns:
        pd.DataFrame: İndirilen veri kümesi.
    """
    # Kaggle veri kümesini indir
    gecici_yol = os.path.join(kagglehub.dataset_download(veri_kumesi_adi), "TurkishSMSCollection.csv")
    return pd.read_csv(gecici_yol, sep=";", encoding="utf-8")

## Veri kümesini oku

In [ ]:
# Veri kümesi bilgileri
dataset_name = "onurkarasoy/turkish-sms-collection"

veri_kumesi = veri_kumesi_oku(dataset_name)
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesinin gereksiz kolonlarını silme

In [ ]:
tohum_degeri = 571
# gereksiz sütunları kaldır
veri_kumesi = veri_kumesi[['Message', 'Group']]
# veri kümesini karıştır
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesini karıştırma

In [ ]:
karismis_veri_kumesi = veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
karismis_veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesini eğitim ve sınama olarak parçalama

In [ ]:
def veri_kumesini_oranli_bol(veri_kumesi, test_orani=0.1, dogrulama_orani=0.1, tohum_degeri=42):
    """
    Veri kümesini, Group kolonundaki değerlerin oranlarını koruyarak üçe böler.

    Args:
        veri_kumesi (pd.DataFrame): Bölünecek veri kümesi
        test_orani (float): Test veri kümesinin oranı (varsayılan: 0.1)
        dogrulama_orani (float): Doğrulama veri kümesinin oranı (varsayılan: 0.1)
        tohum_degeri (int): Random state değeri (tekrarlanabilirlik için)

    Returns:
        tuple: (egitim_veri_kumesi, dogrulama_veri_kumesi, test_veri_kumesi) - Oransal olarak bölünmüş veri kümeleri
    """
    # Benzersiz grup değerlerini bul
    gruplar = veri_kumesi['Group'].unique()

    egitim_liste = []
    dogrulama_liste = []
    test_liste = []

    # Her grup için ayrı ayrı bölme işlemi yap
    for grup in gruplar:
        # Sadece bu gruba ait verileri seç
        grup_verisi = veri_kumesi[veri_kumesi['Group'] == grup]

        # Bu grubu karıştır
        karisik_grup = grup_verisi.sample(frac=1, random_state=tohum_degeri)

        # Bölme noktalarını hesapla
        test_bolme_noktasi = int(len(karisik_grup) * test_orani)
        dogrulama_bolme_noktasi = int(len(karisik_grup) * dogrulama_orani)

        # Bu gruptan test, doğrulama ve eğitim verilerini ayır
        test_grubu = karisik_grup[:test_bolme_noktasi]
        dogrulama_grubu = karisik_grup[test_bolme_noktasi:test_bolme_noktasi+dogrulama_bolme_noktasi]
        egitim_grubu = karisik_grup[test_bolme_noktasi+dogrulama_bolme_noktasi:]

        # Listelere ekle
        test_liste.append(test_grubu)
        dogrulama_liste.append(dogrulama_grubu)
        egitim_liste.append(egitim_grubu)

    # Grupları birleştir
    sinama_veri_kumesi = pd.concat(test_liste).reset_index(drop=True)
    dogrulama_veri_kumesi = pd.concat(dogrulama_liste).reset_index(drop=True)
    egitim_veri_kumesi = pd.concat(egitim_liste).reset_index(drop=True)

    # Son bir karıştırma işlemi
    sinama_veri_kumesi = sinama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    dogrulama_veri_kumesi = dogrulama_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
    egitim_veri_kumesi = egitim_veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)

    return egitim_veri_kumesi, dogrulama_veri_kumesi, sinama_veri_kumesi

In [ ]:
egitim_veri, dogrulama_veri, sinama_veri = veri_kumesini_oranli_bol(karismis_veri_kumesi, tohum_degeri=tohum_degeri)
egitim_veri.head(5)  # İlk 5 satırı göster
dogrulama_veri.head(5)
sinama_veri.head(5)  # İlk 5 satırı göster

## Veri kümelerini eğitime hazır hale getirme fonksiyonu

In [ ]:
# Veri kümesini hazırla
def verileri_hazirla(df, tokenizer):
    # Group değerlerini 0 ve 1'e dönüştür (Group=1 -> 0, Group=2 -> 1)
    # BERT sınıflandırma için 0-based indeksler kullanır
    labels = df["Group"].apply(lambda x: 0 if x == 1 else 1).tolist()

    # Dataset'e dönüştür
    dataset = Dataset.from_dict({
        "text": df["Message"].tolist(),
        "label": labels
    })

    # Tokenize işlemi
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True,
            max_length=512,  # BERT için genellikle 512 kullanılır
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

## Sınama sonucu hesaplama fonksiyonu

In [ ]:
# ölçüt hesaplama fonksiyonu
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Model Eğitim işlemleri

## Model yükleme

In [ ]:
model_adi = "dbmdz/bert-base-turkish-128k-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_adi)
tokenizer = AutoTokenizer.from_pretrained(model_adi)
model.to(cihaz)

In [ ]:
# Eğitim ve test veri kümelerini hazırla
egitim_veri_kumesi = verileri_hazirla(egitim_veri, tokenizer)
dogrulama_veri_kumesi = verileri_hazirla(dogrulama_veri, tokenizer)
sinama_veri_kumesi = verileri_hazirla(sinama_veri, tokenizer)

## Eğitim argümanlarını ve eğiticiyi ayarla

In [ ]:
# Eğitim argümanlarını tanımla
training_args = TrainingArguments(
    output_dir=model_cikti_dizini,
    num_train_epochs=1,  # 1 epoch
    report_to=["tensorboard"],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir=model_log_dizini,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=5,          # Her 5 adımda bir değerlendirme yapılacak
    save_strategy="steps",
    save_steps=5,          # Her 5 adımda bir model kaydedilecek
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # Konsola çıktı için ek parametreler:
    logging_strategy="steps",
    logging_first_step=True,  # İlk adımın da loglanması için
)

# Eğiticiyi oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=egitim_veri_kumesi,
    eval_dataset=dogrulama_veri_kumesi,
    compute_metrics=compute_metrics,
)

## Eğitim ve sınama işlemleri

In [ ]:
# Modeli eğit
trainer.train()

In [ ]:
train_results = trainer.state.log_history
with open(train_sonuc_json_path, "w") as f:
    json.dump(train_results, f, indent=4)

In [ ]:
# Değerlendirme
eval_results = trainer.evaluate(sinama_veri_kumesi)
print(f"Değerlendirme sonuçları: {eval_results}")

with open(eval_sonuc_json_path, "w") as f:
    json.dump(eval_results, f, indent=4)
print(f"Değerlendirme sonuçları {eval_sonuc_json_path} dosyasına kaydedildi.")

## Eğitilen modeli kaydetme

In [ ]:
# Eğitilen modeli kaydet
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Görselleştirme

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Json okuma fonksiyonu

In [ ]:
def json_dosya_oku(dosya_yolu):
    """JSON dosyasını okur ve içeriğini döndürür."""
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        return json.load(f)

## Çizim tarzını ayarla

In [ ]:
plt.style.use('ggplot')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['font.family'] = 'DejaVu Sans'

## Görselleştirme fonksiyonları

In [ ]:
def egitim_sureci_gorsellestirilmesi(train_results, cikti_dizini = model_log_dizini):
    """Eğitim sürecindeki ölçütleri görselleştirir."""
    # Eğitim ve değerlendirme adımlarını ayıkla
    train_steps = [x for x in train_results if 'loss' in x and 'eval_loss' not in x]
    eval_steps = [x for x in train_results if 'eval_loss' in x and 'epoch' in x]
    
    # Grafikler için gerekli verileri hazırla
    epochs_train = [step.get('epoch', 0) for step in train_steps]
    loss_train = [step.get('loss', 0) for step in train_steps]
    
    epochs_eval = [step.get('epoch', 0) for step in eval_steps]
    loss_eval = [step.get('eval_loss', 0) for step in eval_steps]
    accuracy_eval = [step.get('eval_accuracy', 0) for step in eval_steps]
    f1_eval = [step.get('eval_f1', 0) for step in eval_steps]
    precision_eval = [step.get('eval_precision', 0) for step in eval_steps]
    recall_eval = [step.get('eval_recall', 0) for step in eval_steps]
    
    # Kayıp (Loss) Grafiği
    plt.figure(figsize=(14, 8))
    plt.plot(epochs_train, loss_train, 'b-', marker='o', label='Eğitim Kaybı')
    plt.plot(epochs_eval, loss_eval, 'r-', marker='s', label='Doğrulama Kaybı')
    plt.xlabel('Döngü')
    plt.ylabel('Kayıp Değeri')
    plt.title('Eğitim ve Doğrulama Kaybının Döngü Üzerinden Değişimi', fontweight='bold')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'kayip_grafigi.png'), dpi=300, bbox_inches='tight')
    
    # Doğruluk, F1, Kesinlik ve Duyarlılık Grafiği
    plt.figure(figsize=(14, 8))
    plt.plot(epochs_eval, accuracy_eval, 'b-', marker='o', label='Doğruluk')
    plt.plot(epochs_eval, f1_eval, 'g-', marker='s', label='F1 Skoru')
    plt.plot(epochs_eval, precision_eval, 'r-', marker='^', label='Kesinlik')
    plt.plot(epochs_eval, recall_eval, 'm-', marker='D', label='Duyarlılık')
    plt.xlabel('Döngü')
    plt.ylabel('Ölçüt Değeri')
    plt.title('Doğrulama ölçütlerinin Döngü Üzerinden Değişimi', fontweight='bold')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'ölçütler_grafigi.png'), dpi=300, bbox_inches='tight')
    
    # Öğrenme Oranı Grafiği
    if 'learning_rate' in train_steps[0]:
        learning_rates = [step.get('learning_rate', 0) for step in train_steps]
        plt.figure(figsize=(14, 6))
        plt.plot(epochs_train, learning_rates, 'g-', marker='o')
        plt.xlabel('Döngü')
        plt.ylabel('Öğrenme Oranı')
        plt.title('Öğrenme Oranının Döngü Üzerinden Değişimi', fontweight='bold')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'ogrenme_orani_grafigi.png'), dpi=300, bbox_inches='tight')
    plt.close('all')
def degerlendirme_sonuclari_gorsellestirilmesi(eval_sonuc, cikti_dizini = model_log_dizini):
    """Değerlendirme sonuçlarını görselleştirir."""
    # Değerlendirme olcutlerini çıkar
    olcutler = ['accuracy', 'f1', 'precision', 'recall']
    olcut_isimleri = ['Doğruluk', 'F1 Skoru', 'Kesinlik', 'Duyarlılık']
    
    olcut_degerleri = []
    for olcut in olcutler:
        key = f"eval_{olcut}"
        if key in eval_sonuc:
            olcut_degerleri.append(eval_sonuc[key])
        else:
            olcut_degerleri.append(0)  # Eksik olcut değeri için 0
    
    # olcut değerlerini göster
    plt.figure(figsize=(12, 8))
    bars = plt.bar(olcut_isimleri, olcut_degerleri, color=['#3274A1', '#E1812C', '#3A923A', '#C03D3E'])
    plt.ylim(0, 1.1)  # Y ekseni sınırları
    plt.title('Model Değerlendirme Sonuçları', fontweight='bold')
    plt.ylabel('Değer')
    
    # Bar üzerine değerleri ekleme
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.4f}', ha='center', va='bottom', fontsize=12)
    
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(os.path.join(cikti_dizini, 'degerlendirme_sonuclari.png'), dpi=300, bbox_inches='tight')
    
    # Kayıp ve çalışma süresi grafiği
    if 'eval_loss' in eval_sonuc and 'eval_runtime' in eval_sonuc:
        plt.figure(figsize=(10, 6))
        plt.bar(['Kayıp', 'Çalışma Süresi (sn)'], 
                [eval_sonuc['eval_loss'], eval_sonuc['eval_runtime']], 
                color=['#C03D3E', '#3274A1'])
        plt.title('Değerlendirme Kaybı ve Çalışma Süresi', fontweight='bold')
        plt.grid(axis='y')
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'kayip_ve_sure.png'), dpi=300, bbox_inches='tight')
    plt.close('all')
def egitim_ozet_tablosu(train_results, eval_sonuc, cikti_dizini = model_log_dizini):
    """Eğitim sürecinin özet tablosunu oluşturur ve kaydeder."""
    # Son eğitim Ölçütleri
    son_egitim = None
    for step in reversed(train_results):
        if 'train_runtime' in step:
            son_egitim = step
            break
    
    if son_egitim:
        # Tablo verilerini hazırla
        data = {
            'Ölçüt': ['Eğitim Süresi (sn)', 'Ör./sn', 'Adım/sn', 'Toplam FLOPS', 
                      'Son Kayıp', 'Son Döngü', 'Doğruluk', 'F1 Skoru'],
            'Değer': [
                son_egitim.get('train_runtime', 'N/A'),
                son_egitim.get('train_samples_per_second', 'N/A'),
                son_egitim.get('train_steps_per_second', 'N/A'),
                f"{son_egitim.get('total_flos', 0) / 1e12:.2f} T" if 'total_flos' in son_egitim else 'N/A',
                son_egitim.get('train_loss', 'N/A'),
                son_egitim.get('epoch', 'N/A'),
                eval_sonuc.get('eval_accuracy', 'N/A'),
                eval_sonuc.get('eval_f1', 'N/A')
            ]
        }
        
        # DataFrame oluştur
        df = pd.DataFrame(data)
        
        # Tabloyu görselleştir
        plt.figure(figsize=(10, 6))
        table = plt.table(cellText=df.values, colLabels=df.columns, 
                         loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1, 1.5)
        plt.axis('off')
        plt.title('Eğitim Özet Tablosu', fontweight='bold', y=0.9)
        plt.tight_layout()
        plt.savefig(os.path.join(cikti_dizini, 'egitim_ozeti.png'), dpi=300, bbox_inches='tight')
        plt.close('all')

## Sonuçları oku ve görselleştir

In [ ]:
train_results = json_dosya_oku(train_sonuc_json_path)
eval_sonuc = json_dosya_oku(eval_sonuc_json_path)

In [ ]:
# Görselleştirmeleri oluştur
print("Eğitim süreci görselleştiriliyor...")
egitim_sureci_gorsellestirilmesi(train_results)

print("Değerlendirme sonuçları görselleştiriliyor...")
degerlendirme_sonuclari_gorsellestirilmesi(eval_sonuc)

print("Eğitim özet tablosu oluşturuluyor...")
egitim_ozet_tablosu(train_results, eval_sonuc)